In [6]:
from qmratool.models import *
#import pymc3 as pm
import plotly.express as px
#import arviz as az
import numpy as np
import pandas as  pd
import plotly.express as px
from plotly.offline import plot
from django_pandas.io import read_frame
import decimal

In [7]:
def annual_risk(nexposure, event_probs):
    return 1-np.prod(1-np.random.choice(event_probs, nexposure, True))


In [102]:
def simulate_risk(ra):
    # Selecting inflow concentration based in source water type
    df_inflow = read_frame(Inflow.objects.filter(water_source=ra.source).values("min", "max", "pathogen__pathogen", "water_source__water_source_name"))
    df_inflow = df_inflow[df_inflow["pathogen__pathogen"].isin(["Rotavirus", "Campylobacter jejuni", "Cryptosporidium parvum"])]
    # Querying dose response parameters based on pathogen inflow
    dr_models = read_frame(DoseResponse.objects.filter(pathogen__in=Pathogen.objects.filter(pathogen__in=df_inflow["pathogen__pathogen"])))

    # Querying for Logremoval based on selected treatments
    df_treatment=read_frame(LogRemoval.objects.filter(treatment__in=ra.treatment.all()).values("min", "max", "treatment__name", "pathogen_group__pathogen_group"))
    
    # Summarizing treatment to treatment max and treatment min
    df_treatment_summary=df_treatment.groupby(["pathogen_group__pathogen_group"]).sum().reset_index()

    results = pd.DataFrame()

    for index, row in df_inflow.iterrows():
        d = df_inflow.loc[df_inflow["pathogen__pathogen"] ==row["pathogen__pathogen"]]
        dr = dr_models.loc[dr_models["pathogen"]==row["pathogen__pathogen"]]

        if row["pathogen__pathogen"] == "Rotavirus":
            selector = "Viruses" 
        elif row["pathogen__pathogen"]== "Cryptosporidium parvum":
            selector = "Protozoa"
        else:
            selector = "Bacteria"
        #result.append(selector)

        df_treat = df_treatment_summary[df_treatment_summary["pathogen_group__pathogen_group"]==selector]



        risk_df = pd.DataFrame({"inflow": np.random.normal(loc=(np.log10(float(d["min"])+10**(-8))+np.log10(float(d["max"]) ))/2, 
                                                            scale = (np.log10(float(d["max"]))-np.log10(float(d["min"])+10**(-8) ))/4,  
                                                            size = 10000),
                                "LRV": np.random.uniform(low= df_treat["min"], 
                                                         high= df_treat["max"], 
                                                         size= 10000) })
        risk_df["outflow"]=risk_df["inflow"] - risk_df["LRV"]
        risk_df["dose"] = (10**risk_df["outflow"])*float(ra.exposure.volume_per_event)

        if selector != "Protozoa":
            risk_df["probs"] = 1 - (1 + (risk_df["dose"]) * (2 ** (1/float(dr.alpha)) - 1)/float(dr.n50)) ** -float(dr.alpha)
        else:
            risk_df["probs"] = 1 - np.exp(-float(dr.k)*(risk_df["dose"]))

        results[row["pathogen__pathogen"]] = [annual_risk(int(ra.exposure.events_per_year), risk_df["probs"] ) for _ in range(1000)]

    results_long = pd.melt(results)
    results_long["log probability"] = np.log10(results_long["value"])
    return results_long

In [106]:
ras = RiskAssessment.objects.all()[:5]

In [107]:
results = []
for i in range(len(ras)):
    sim = simulate_risk(ras[i])
    sim["Assessment"] = ras[i].name
    results.append(sim)
  
df = pd.concat(results)

In [116]:
fig = px.box(df, x="variable", y="value", color="Assessment", 
                            points="all",  log_y =True, 
                            title="Risk as probability of infection per year",
                            color_discrete_sequence=["#004254", "#007c9e", "#a3d1ec","#3494ae","#00B8eb"])


fig.update_layout(
    font_family="Helvetica Neue, Helvetica, Arial, sans-serif",
    font_color="black",
    title = {'text':'Risk assessment as probability of infection per year'},
    xaxis_title = "Reference Pathogen",
    yaxis_title = "Probability of infection per year",
    #markersize= 12,
    )

fig.update_layout(legend=dict(
                 orientation="h",
                 yanchor="top",
                 y=-.1,
                 xanchor="left",
                x=0))


fig.update_traces(marker_size = 8)#['#75c3ff', "red"],#, marker_line_color='#212c52',


fig.show()

